In [2]:
!pip install efficientnet

     |████████████████████████████████| 50 kB 8.3 MB/s 


In [3]:
!pip install tensorflow-addons

     |████████████████████████████████| 1.1 MB 29.8 MB/s 


In [4]:
import os  # 폴더 경로 잡기 위해 필요
import numpy as np
from tensorflow.keras.preprocessing.image import ImageDataGenerator

import efficientnet
import efficientnet.tfkeras as efn

from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Flatten, Dense, Dropout, Input

from tensorflow.keras.layers import GlobalAveragePooling2D  # efficientnet 사용시 좋음
import tensorflow_addons as tfa
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping

from tensorflow.keras.optimizers import RMSprop, Adam
import matplotlib.pyplot as plt
import cv2 as cv    # opencv(이미지 resize)

In [5]:
train_dir = '/content/drive/MyDrive/Colab임시폴더/cat_dog_small/train'
valid_dir = '/content/drive/MyDrive/Colab임시폴더/cat_dog_small/validation'

IMAGE_SIZE = 256
BATCH_SIZE = 8
LEARNING_RATE = 5e-5

# ImageDataGenerator를 위한 전처리 함수
# 여기서의 역할은 이미지 resize
def generate_preprocessing(img):
    # 여러가지 전처리 작업이 포함될 수 있다!
    img = cv.resize(img, (IMAGE_SIZE, IMAGE_SIZE))
    return img

# ImageDataGenerator부터 만들자!
# train data는 augmentation(증식) 해서 생성할 것이다!
train_datagen = ImageDataGenerator(rescale=1/255,
                                   rotation_range=40,
                                   width_shift_range=0.1,
                                   height_shift_range=0.1,
                                   zoom_range=0.2,
                                   horizontal_flip=True,
                                   vertical_flip=True,
                                   preprocessing_function=generate_preprocessing,
                                   # 이미지 전처리 함수 포함 가능
                                   fill_mode='nearest')

valid_datagen = ImageDataGenerator(rescale=1/255,
                                   preprocessing_function=generate_preprocessing)

train_generator = train_datagen.flow_from_directory(
    train_dir,
    classes=['cats', 'dogs'],
    batch_size=BATCH_SIZE,
    class_mode='binary'
)

valid_generator = valid_datagen.flow_from_directory(
    valid_dir,
    classes=['cats', 'dogs'],
    batch_size=BATCH_SIZE,
    class_mode='binary',
    shuffle=False   # default가 True로 데이터를 섞어서 가져온다.
)



Found 2000 images belonging to 2 classes.
Found 1000 images belonging to 2 classes.


In [11]:
# Pretrained Network

pretrained_network = efn.EfficientNetB4(
    weights='imagenet',
    include_top=False,
    input_shape=(IMAGE_SIZE,IMAGE_SIZE,3)
)

# print(pretrained_network.summary())
# for idx, layer in enumerate(pretrained_network.layers):
#     # layers는 layer들을 list형태로 가지고 있다.
#     print('{}번 : {}'.format(idx,layer.name))

pretrained_network.trainable = False  # 학습이 안되게끔 동결!!

model = Sequential()

model.add(pretrained_network)

# FC layer 나오기 전에 GlobalAveragePooling2D가 나오는 게 좋다는 논문이 있다.
# Input layer 역할을 한다.
model.add(GlobalAveragePooling2D())
model.add(Dense(units=1,
                activation='sigmoid'))

# Early Stopping
es = EarlyStopping(monitor='val_loss',
                   mode='auto',
                   patience=5,  # 5번 연속으로 val_loss 값이 안좋아지면 stop
                   verbose=1)

# Checkpoint
model_checkpoint = './{epoch:06d}-{val_acc:0.6f}-{acc:0.6f}.ckpt'
                    # epoch 수, val_acc 값, acc(train) 값을 파일명으로 생성
checkpointer = ModelCheckpoint(
    filepath=model_checkpoint,
    verbose=1,
    period=2,  # 2 epoch당 파일이 하나씩 만들어진다.
    save_best_weights=True,
    mode='auto',
    monitor='val_acc'
)

model.compile(optimizer=Adam(learning_rate=LEARNING_RATE),
              loss='binary_crossentropy',
              metrics=['acc'])  # 위에 파일명 설정 때문에 이렇게 잡아준다.

history = model.fit(train_generator,
                    steps_per_epoch=(2000 // BATCH_SIZE),
                    epochs=30,
                    validation_data=valid_generator,
                    validation_steps=(1000 // BATCH_SIZE),
                    callbacks=[es, checkpointer],  # Early Stopping, Checkpoint
                    verbose=1)


Epoch 1/30
250/250 [==============================] - 651s 3s/step - loss: 0.6459 - acc: 0.6370 - val_loss: 0.5188 - val_acc: 0.8430
Epoch 2/30
250/250 [==============================] - ETA: 0s - loss: 0.5132 - acc: 0.8135
Epoch 2: saving model to ./000002-0.908000-0.813500.ckpt


INFO:tensorflow:Assets written to: ./000002-0.908000-0.813500.ckpt/assets


INFO:tensorflow:Assets written to: ./000002-0.908000-0.813500.ckpt/assets


250/250 [==============================] - 91s 366ms/step - loss: 0.5132 - acc: 0.8135 - val_loss: 0.3969 - val_acc: 0.9080
Epoch 3/30
250/250 [==============================] - 47s 187ms/step - loss: 0.4438 - acc: 0.8605 - val_loss: 0.3240 - val_acc: 0.9310
Epoch 4/30
250/250 [==============================] - ETA: 0s - loss: 0.3968 - acc: 0.8700
Epoch 4: saving model to ./000004-0.944000-0.870000.ckpt


INFO:tensorflow:Assets written to: ./000004-0.944000-0.870000.ckpt/assets


...

INFO:tensorflow:Assets written to: ./000028-0.987000-0.928000.ckpt/assets


250/250 [==============================] - 92s 369ms/step - loss: 0.1956 - acc: 0.9280 - val_loss: 0.0771 - val_acc: 0.9870
Epoch 29/30
250/250 [==============================] - 47s 187ms/step - loss: 0.1855 - acc: 0.9255 - val_loss: 0.0754 - val_acc: 0.9870
Epoch 30/30
250/250 [==============================] - ETA: 0s - loss: 0.1873 - acc: 0.9325
Epoch 30: saving model to ./000030-0.987000-0.932500.ckpt


INFO:tensorflow:Assets written to: ./000030-0.987000-0.932500.ckpt/assets


INFO:tensorflow:Assets written to: ./000030-0.987000-0.932500.ckpt/assets


250/250 [==============================] - 92s 370ms/step - loss: 0.1873 - acc: 0.9325 - val_loss: 0.0742 - val_acc: 0.9870


In [ ]:
# Pretrained Network의 layer 몇개를 학습이 가능한 형태로
# 동결을 풀고
# 재학습. (fine tuning)